In [21]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adagrad
from keras.losses import MeanSquaredError, MeanAbsoluteError, Huber, LogCosh

In [3]:
temp = pd.read_csv('./data/temp.csv')
stock = pd.read_csv('./data/TSLA.csv')

In [4]:
stock = stock.iloc[:1000]
stock = stock[['close', 'time']]
stock['time'] = pd.to_datetime(stock['time'])
stock = stock.set_index('time')

temp = temp[['date', 'meantemp']]
temp['date'] = pd.to_datetime(temp['date'])
temp = temp.set_index('date')

In [5]:
train_size = int(len(stock) * 0.8)
train, test = stock[0:train_size], stock[train_size:len(stock)]

In [22]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=1))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [23]:
losses = [MeanSquaredError(), MeanAbsoluteError(), Huber(), LogCosh()]
optimizers = [Adam(), SGD(), RMSprop(), Adagrad()]

In [24]:
best_loss = float('inf')
best_model = None

best_loss_name = None
best_optim_name = None


for loss in losses:
    for optimizer in optimizers:
        model.compile(loss=loss, optimizer=optimizer)
        model.fit(train, train, epochs=50, verbose=0)
        
        # Оценка модели
        test_loss = model.evaluate(test, test, verbose=0)
        if test_loss < best_loss:
            best_loss = test_loss
            best_model = model
            

print("Лучшая модель имеет loss:", best_loss)

Лучшая модель имеет loss: 0.07295605540275574


In [26]:
best_model.optimizer, best_model.loss

(<keras.src.optimizers.adagrad.Adagrad at 0x1a848e91390>,
 <keras.src.losses.LogCosh at 0x1a849f6dd80>)

In [27]:
train_size = int(len(temp) * 0.8)
train, test = temp[0:train_size], temp[train_size:len(temp)]

In [28]:
best_loss = float('inf')
best_model = None

best_loss_name = None
best_optim_name = None


for loss in losses:
    for optimizer in optimizers:
        model.compile(loss=loss, optimizer=optimizer)
        model.fit(train, train, epochs=50, verbose=0)
        
        # Оценка модели
        test_loss = model.evaluate(test, test, verbose=0)
        if test_loss < best_loss:
            best_loss = test_loss
            best_model = model
            

print("Лучшая модель имеет loss:", best_loss)

Лучшая модель имеет loss: 5.1132988929748535


In [29]:
best_model.optimizer, best_model.loss

(<keras.src.optimizers.adagrad.Adagrad at 0x1a848e91390>,
 <keras.src.losses.LogCosh at 0x1a849f6dd80>)

## На обоих наборах данных лучшими покуказали себя Adagrad и LogCosh

* Adagrad в целом считается самым гибким и лучшим оптимизатором, который стоит использовать в 99% случаев, поэтому данный результат очевиден
* Опыта работы ранее с LogCosh не имел, поэтому выводы делаю исключительно из теоретического материала. Во первых, LogCosh менее чувствительна к выбросам, чем другие метрики, что для данных котировок очень важно. Также имеет особенность к сглаживанию ошибок, а также устойчива численно, то есть не уходит так быстро в бесконечность, как например MSE. Также LogCosh дифференцируема